In [1]:
install.packages("plm")
install.packages("AER")


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘miscTools’, ‘rbibutils’, ‘bdsmatrix’, ‘collapse’, ‘zoo’, ‘sandwich’, ‘lmtest’, ‘maxLik’, ‘Rdpack’, ‘Formula’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘cowplot’, ‘Deriv’, ‘microbenchmark’, ‘numDeriv’, ‘doBy’, ‘SparseM’, ‘MatrixModels’, ‘minqa’, ‘nloptr’, ‘RcppEigen’, ‘carData’, ‘abind’, ‘pbkrtest’, ‘quantreg’, ‘lme4’, ‘car’




In [18]:

library(plm)
library(AER)
library(dplyr)
library("readxl")

data <- read_excel('/content/dataset2.xlsx')
data2 <- read_excel('/content/dataset3.xlsx')

In [11]:
install.packages("sandwich")
install.packages("lmtest")

library(sandwich)
library(lmtest)


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [13]:
install.packages("fixest")
library(fixest)

install.packages("modelsummary")
library(modelsummary)


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘dreamerr’, ‘stringmagic’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘bayestestR’, ‘datawizard’, ‘checkmate’, ‘insight’, ‘parameters’, ‘performance’, ‘tables’, ‘tinytable’


`modelsummary` 2.0.0 now uses `tinytable` as its default table-drawing
  backend. Learn more at: https://vincentarelbundock.github.io/tinytable/

Revert to `kableExtra` for one session:

  options(modelsummary_factory_default = 'kableExtra')
  options(modelsummary_factory_latex = 'kableExtra')
  options(modelsummary_factory_html = 'kableExtra')

Silence this message forever:

  config_modelsummary(startup_message = FALSE)



In [14]:
colnames(data)

[1] "district_nhood"          "district"               
 [3] "border_district_nhood"   "border_district"        
 [5] "hhold_recipient"         "population2023"         
 [7] "SES_numeric"             "house_price"            
 [9] "cumhur"                  "eko_2019_june_r"        
[11] "log_population"          "log_house_price"        
[13] "county_pair_dummy"       "hhold_recipient_box_cox"

In [19]:
#Within Marginals
eko1 <- lm(hhold_recipient_box_cox ~ eko_2019_june_r + factor(county_pair_dummy), data = data)

clustered_se_eko1 <- vcovCL(eko1, cluster = ~ county_pair_dummy)

eko2 <- lm(hhold_recipient_box_cox ~ eko_2019_june_r + log_house_price + log_population +   factor(county_pair_dummy), data = data)

clustered_se_eko2 <- vcovCL(eko2, cluster = ~ county_pair_dummy)



In [20]:
#Table5
model_list <- list(eko1, eko2)
vcov_list <- list(clustered_se_eko1, clustered_se_eko2)

custom_stars <- c('*' = 0.10, '**' = 0.05, '***' = 0.01)

modelsummary(model_list,
            vcov = vcov_list,
            stars = custom_stars,
            coef_omit = "factor\\(county_pair_dummy\\)",
            gof_omit = 'Within|Std.Errors|AIC|BIC|RMSE',
            fmt = 3,
            notes = "The standard errors are clustered at the county pair level.")  # Add custom footnote for significance stars


+-----------------+-----------+-----------+
|                 | (1)       | (2)       |
+=================+===========+===========+
| (Intercept)     | 22.587*** | 7.875     |
+-----------------+-----------+-----------+
|                 | (1.887)   | (6.344)   |
+-----------------+-----------+-----------+
| eko_2019_june_r | -6.269    | -5.356*   |
+-----------------+-----------+-----------+
|                 | (4.583)   | (2.813)   |
+-----------------+-----------+-----------+
| log_house_price |           | -2.919*** |
+-----------------+-----------+-----------+
|                 |           | (0.551)   |
+-----------------+-----------+-----------+
| log_population  |           | 4.426***  |
+-----------------+-----------+-----------+
|                 |           | (0.247)   |
+-----------------+-----------+-----------+
| Num.Obs.        | 1344      | 1344      |
+-----------------+-----------+-----------+
| R2              | 0.849     | 0.945     |
+-----------------+-----------+

In [21]:
data2$swing_dummy <- ifelse(data2$eko_2019_june_r >= 0.4 & data2$eko_2019_june_r <= 0.6, 1, 0)

In [22]:
colnames(data2)

[1] "district_nhood"          "district"               
 [3] "border_district_nhood"   "border_district"        
 [5] "hhold_recipient"         "population2023"         
 [7] "SES_numeric"             "house_price"            
 [9] "cumhur"                  "eko_2019_june_r"        
[11] "log_population"          "log_house_price"        
[13] "county_pair_dummy"       "hhold_recipient_box_cox"
[15] "swing_dummy"

In [25]:
#Marginal vs Others
eko1v2 <- lm(hhold_recipient_box_cox ~ swing_dummy + factor(county_pair_dummy), data = data2)

eko2v2 <- lm(hhold_recipient_box_cox ~ swing_dummy + log_house_price+ log_population + factor(county_pair_dummy) , data = data2)


cluster_se_eko1v2 <- vcovCL(eko1v2, cluster = ~ county_pair_dummy)
cluster_se_eko2v2 <- vcovCL(eko2v2, cluster = ~ county_pair_dummy)



In [26]:
#Table 5
model_list <- list(eko1v2, eko2v2)
vcov_list <- list(cluster_se_eko1v2, cluster_se_eko2v2)


custom_stars <- c('*' = 0.10, '**' = 0.05, '***' = 0.01)

modelsummary(model_list,
            vcov = vcov_list,
            stars = custom_stars,
            coef_omit = "factor\\(county_pair_dummy\\)",
            gof_omit = 'Within|Std.Errors|AIC|BIC|RMSE',
            fmt = 3,
            notes = "The standard errors are clustered at the county pair level.")  # Add custom footnote for significance stars


+-----------------+-----------+-----------+
|                 | (1)       | (2)       |
+=================+===========+===========+
| (Intercept)     | 12.795*** | 14.643**  |
+-----------------+-----------+-----------+
|                 | (0.158)   | (6.168)   |
+-----------------+-----------+-----------+
| swing_dummy     | 0.705**   | 0.553***  |
+-----------------+-----------+-----------+
|                 | (0.315)   | (0.201)   |
+-----------------+-----------+-----------+
| log_house_price |           | -2.997*** |
+-----------------+-----------+-----------+
|                 |           | (0.641)   |
+-----------------+-----------+-----------+
| log_population  |           | 2.892***  |
+-----------------+-----------+-----------+
|                 |           | (0.174)   |
+-----------------+-----------+-----------+
| Num.Obs.        | 768       | 768       |
+-----------------+-----------+-----------+
| R2              | 0.844     | 0.938     |
+-----------------+-----------+